In [3]:
cd ~/ceph/DATA/CRISPR/crispr_kinn/

/mnt/ceph/users/alamson/DATA/CRISPR/crispr_kinn


In [4]:
import pandas as pd
from tqdm import tqdm
from Bio import pairwise2
from Bio.Seq import Seq
from src.crispr_kinn_predict import (reload_from_dir, 
                                     KineticNeuralNetworkBuilder, 
                                     featurize_alignment, get_letter_index)
import tensorflow as tf
import numpy as np

2022-11-02 12:12:09.593405: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-02 12:12:09.764400: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-11-02 12:12:09.768177: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /mnt/sw/nix/store/f0ycdncw8dw4wlicnzm74lgv9c51rlg4-openblas-0.3.20/lib:/cm/shared/a

detected tf2 - using compatibility mode


In [5]:
from pyfaidx import Fasta
genes = Fasta('/mnt/ceph/users/zzhang/genome_assembly/GRCh38.primary_assembly.genome.fa')
def add_seq_to_bed(bed_file):
    gene_bed = pd.read_table(bed_file)
    gene_bed.head(10)
    seq_list = []
    for i in range(gene_bed.shape[0]):
        gene_pam = gene_bed.iloc[i]
        prev_num = gene_pam['num']
        if gene_pam["strand"] == '-':
            fasta_gene = genes[gene_pam["chrom"]][gene_pam['start']:gene_pam['end']].reverse.complement
        else:
            fasta_gene = genes[gene_pam["chrom"]][gene_pam['start']-1:gene_pam['end']-1]
            
        seq_list += [str(fasta_gene)]

    gene_bed['seq'] = seq_list
    gene_bed['PamID'] = gene_bed['pamid']
    return gene_bed[['chrom', 'start', 'end', 'strand', 'PamID', 'genename', 'num', 'seq']].drop_duplicates(subset=['num'], keep='first')

In [12]:
gene = add_seq_to_bed('/mnt/ceph/users/alamson/DATA/CRISPR/genes/PDCD1.bed')

In [13]:
grna_seq = []
for i in range(gene.shape[0]):
    # 20 gRNA + 3 PAM
    grna_seq.append(gene.iloc[i]['seq'][13:36])
gene['grna_seq'] = grna_seq
print(grna_seq)

['TGGCCAAGGAAGCCGGTCAGAGG', 'GGCCAAGGAAGCCGGTCAGAGGG', 'GCCAAGGAAGCCGGTCAGAGGGG', 'CTGAGGATGGACACTGCTCTTGG', 'CAGCCACTGAGGCCTGAGGATGG', 'CAAGAGCAGTGTCCATCCTCAGG', 'TGCCCAGCCACTGAGGCCTGAGG', 'TGTCCATCCTCAGGCCTCAGTGG', 'CTCGGAGTGCCCAGCCACTGAGG', 'CATCCTCAGGCCTCAGTGGCTGG', 'ATCCTCAGGCCTCAGTGGCTGGG', 'CTCAGTGGCTGGGCACTCCGAGG', 'TCAGTGGCTGGGCACTCCGAGGG', 'GGGGCTCAGCTGACGGCCCTCGG', 'GCCCGCAGGGGCTCAGCTGACGG', 'GGCCGTCAGCTGAGCCCCTGCGG', 'GCCGTCAGCTGAGCCCCTGCGGG', 'ACCTCATCCCCCGCCCGCAGGGG', 'CACCTCATCCCCCGCCCGCAGGG', 'GTCAGCTGAGCCCCTGCGGGCGG', 'GCACCTCATCCCCCGCCCGCAGG', 'TCAGCTGAGCCCCTGCGGGCGGG', 'CAGCTGAGCCCCTGCGGGCGGGG', 'AGCTGAGCCCCTGCGGGCGGGGG', 'GCCCCTGCGGGCGGGGGATGAGG', 'ATTGTCTTTCCTAGCGGAATGGG', 'CATTGTCTTTCCTAGCGGAATGG', 'GATGAGGTGCCCATTCCGCTAGG', 'GCCACCATTGTCTTTCCTAGCGG', 'CATTCCGCTAGGAAAGACAATGG', 'TCCGCTAGGAAAGACAATGGTGG', 'GCCCTGTGTCCCTGAGCAGACGG', 'GTGGCATACTCCGTCTGCTCAGG', 'TGGCATACTCCGTCTGCTCAGGG', 'CTCCGTCTGCTCAGGGACACAGG', 'TCCGTCTGCTCAGGGACACAGGG', 'CTGCTCAGGGACACAGGGCACGG', 

In [14]:
# sanity check
for _ in grna_seq:
    assert _.endswith('GG')

In [15]:
gene.shape

(211, 9)

In [17]:
n_mismatch = 6
n_bulges = 1
with open("CasOffinderPDCD1_input.txt", "w") as f:
    # first line: FASTA filepath
    f.write("/mnt/ceph/users/zzhang/genome_assembly/GRCh38.primary_assembly.genome.fa\n")
    # second line: desired pattern including PAM site and optional DNA or RNA bulge sizes, separated by spaces
    f.write(f"{'N'*20}NGG {n_bulges} {n_bulges}\n")
    # remaining lines: the query sequences and maximum mismatch numbers, separated by spaces
    for i in range(gene.shape[0]):
        f.write(f"{gene.iloc[i]['grna_seq']} {n_mismatch}\n")

In [19]:
%%time
%%bash
cas-offinder CasOffinderPDCD1_input.txt C CasOffinderPDCD1_output.txt

Total 1 device(s) found.
Loading input file...
Reading /mnt/ceph/users/zzhang/genome_assembly/GRCh38.primary_assembly.genome.fa...
Sending data to devices...
Chunk load started.
1 devices selected to analyze...
Finding pattern in chunk #1...
Comparing patterns in chunk #1...
530.957 seconds elapsed.


CPU times: user 5.71 ms, sys: 16.4 ms, total: 22.1 ms
Wall time: 8min 51s


In [22]:
cas_off_ver = '2.4'

if cas_off_ver == '2.4':
    ot = pd.read_table("CasOffinderPDCD1_output.txt",
                      names=['grna_seq', "chrom", "pos", "ot_seq", "strand", "mm"])
    ot['chrom'] = [x.split()[0] for x in ot['chrom']]
    ot = ot.query('mm>0').reset_index()
else:
    ot = pd.read_table("CasOffinderPDCD1_output.txt", skiprows=1, low_memory=False)
    ot = ot.query('Mismatches>0').reset_index() #for V3.0
    # fix bulge
    for i, row in tqdm(ot.iterrows(), total=ot.shape[0]):
        if row['crRNA'].startswith('-'):
            ot.iloc[i]['crRNA'] = ot.iloc[i]['crRNA'].lstrip('-')
            ot.iloc[i]['DNA'] = ot.iloc[i]['DNA'][1:]
        if row['crRNA'].endswith('-'):
            ot.iloc[i]['crRNA'] = ot.iloc[i]['crRNA'].lstrip('-')
            ot.iloc[i]['DNA'] = ot.iloc[i]['DNA'][:-1]

In [23]:
print(ot.shape)
ot.head()

(1395022, 7)


,index,grna_seq,chrom,pos,ot_seq,strand,mm
0,0,TGGCCAAGGAAGCCGGTCAGAGG,chr4,98389191,TGtCCctGGAAcCCttTCAGAGGATCA,-,6
1,1,TGGCCAAGGAAGCCGGTCAGAGG,chr2,58670020,TGcCCAgaGtAGCaGcTCAGAGGCAGT,+,6
2,2,TGGCCAAGGAAGCCGGTCAGAGG,chr5,163383891,TGGaCAAGttAGCCaGTCAGAGGCTTT,-,4
3,3,TGGCCAAGGAAGCCGGTCAGAGG,chr2,20618941,TGGCCcAGcAgGCCtGTCtGtGGGGAC,-,6
4,4,TGGCCAAGGAAGCCGGTCAGAGG,chr4,9811091,TGaCCAAGctgGCCaaTCAGAGGAACT,+,6


In [24]:
if cas_off_ver == '2.4':
    ### if using Cas-OFFinder v2.4
    alignments = []
    maxlen = 25
    for _, row in tqdm(ot.iterrows(), total=ot.shape[0]):
        ref = Seq(row['grna_seq'][::-1])
        alt = Seq(row['ot_seq'].upper()[0:23][::-1])
        # m: A match score is the score of identical chars, otherwise mismatch score
        # d: The sequences have different open and extend gap penalties.
        aln = pairwise2.align.localxd(ref, alt, -1, -0.1, -1, 0)
        if len(aln[0][0]) > maxlen: # increase gap open penalty to avoid too many gaps
            aln = pairwise2.align.localxd(ref, alt, -5, -0.1, -5, 0)
            if len(aln[0][0]) > maxlen:
                aln = [(ref, alt)]
        alignments.append(aln[0])
    alignment_df = pd.DataFrame({'ref':[x[0] for x in alignments], 'alt':[x[1] for x in alignments]})
elif cas_off_ver == '3.0':
    ### if using Cas-OFFinder v3.0
    alignments = [x[1].str[::-1].str.upper().tolist() for x in tqdm(ot[['crRNA', 'DNA']].iterrows(), total=ot.shape[0])]
    alignment_df = pd.DataFrame({'ref':[x[0] for x in alignments], 'alt':[x[1] for x in alignments]})


100%|██████████| 1395022/1395022 [09:00<00:00, 2581.67it/s]


In [25]:
# featurize alignment
ltidx = get_letter_index(build_indel=True)
fea = featurize_alignment(alignments, ltidx)

In [33]:
# load kinn
sess = tf.compat.v1.Session()
manager_kwargs={
    'output_op': 
        lambda: tf.keras.layers.Lambda(lambda x: tf.math.log(x)/np.log(10), name="output_log"),
    'n_feats': 25,  # remember to change this!!
    'n_channels': 9,
    'batch_size': 128,
    'epochs': 30,
    'earlystop': 10,
    'verbose': 0
}
kinn_1 = reload_from_dir(wd="outputs/2022-05-21/KINN-wtCas9_cleave_rate_log-finkelstein-0-rep4-gRNA1/",
                         sess=sess,
                         manager_kwargs=manager_kwargs,
                         model_fn=KineticNeuralNetworkBuilder
                         )
kinn_2 = reload_from_dir(wd="outputs/2022-05-21/KINN-wtCas9_cleave_rate_log-finkelstein-0-rep5-gRNA2/",
                         sess=sess,
                         manager_kwargs=manager_kwargs,
                         model_fn=KineticNeuralNetworkBuilder
                         )

2022-11-02 14:16:26.918272: W tensorflow/c/c_api.cc:291] Operation '{name:'conv_k4_2/bias/Assign' id:625 op device:{requested: '', assigned: ''} def:{{{node conv_k4_2/bias/Assign}} = AssignVariableOp[_has_manual_control_dependencies=true, dtype=DT_FLOAT, validate_shape=false](conv_k4_2/bias, conv_k4_2/bias/Initializer/zeros)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't modify nodes after running them or create a new session.
2022-11-02 14:16:27.226157: W tensorflow/c/c_api.cc:291] Operation '{name:'conv_k3_3/bias/Assign' id:860 op device:{requested: '', assigned: ''} def:{{{node conv_k3_3/bias/Assign}} = AssignVariableOp[_has_manual_control_dependencies=true, dtype=DT_FLOAT, validate_shape=false](conv_k3_3/bias, conv_k3_3/bias/Initializer/zeros)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in th

In [34]:
# build rate model
l1 = {l.name:l for l in kinn_1.model.layers}
kinn_1_rate_mod = tf.keras.Model(
    inputs=kinn_1.model.inputs,
    outputs=l1['gather_rates'].output
)
l2 = {l.name:l for l in kinn_2.model.layers}
kinn_2_rate_mod = tf.keras.Model(
    inputs=kinn_2.model.inputs,
    outputs=l2['gather_rates'].output
)

In [35]:
k1_clv = kinn_1.predict(fea)
k2_clv = kinn_2.predict(fea)
k1_rates = np.array(kinn_1_rate_mod.predict(kinn_1.blockify_seq_ohe(fea)))
k2_rates = np.array(kinn_2_rate_mod.predict(kinn_2.blockify_seq_ohe(fea)))

/mnt/home/alamson/local/envs/kinn/lib/python3.8/site-packages/keras-2.11.0rc2-py3.8.egg/keras/engine/training_v1.py:2357: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,
2022-11-02 14:16:32.092471: W tensorflow/c/c_api.cc:291] Operation '{name:'output_log_2/truediv' id:701 op device:{requested: '', assigned: ''} def:{{{node output_log_2/truediv}} = RealDiv[T=DT_FLOAT, _has_manual_control_dependencies=true](output_log_2/Log, output_log_2/truediv/y)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't modify nodes after running them or create a new session.
2022-11-02 14:17:33.741198: W tensorflow/c/c_api.cc:291] Operation '{name:'output_log_3/truediv' id:959 op device:{requested: '', assigned: ''} def:{{{node output_log_3/truediv}} = RealDiv[

In [36]:
k1_kinn = pd.DataFrame(np.hstack([k1_clv, k1_rates]), 
                       columns=['pred_cleavage_log10', 'k_on_log', 'k_off_log', 'k_OI_log', 
                                'k_IO_log', 'k_IC_log', 'k_CI_log', 'k_cat_log'])
k2_kinn = pd.DataFrame(np.hstack([k2_clv, k2_rates]), 
                       columns=['pred_cleavage_log10', 'k_on_log', 'k_off_log', 'k_OI_log', 
                                'k_IO_log', 'k_IC_log', 'k_CI_log', 'k_cat_log'])
kinn = k1_kinn.join(k2_kinn, lsuffix='.1', rsuffix='.2')

In [37]:
ot_res = ot.join(alignment_df).join(kinn)

In [38]:
ot_res['pred_cleavage_log10'] = ot_res[['pred_cleavage_log10.1', 'pred_cleavage_log10.2']].mean(axis=1)

In [39]:
ot_res.shape

(1395022, 26)

In [40]:
ot_res = ot_res.query('pred_cleavage_log10 > -8')

In [41]:
ot_res_w_on = ot_res.set_index('grna_seq').join(gene.set_index('grna_seq'), lsuffix='.off', rsuffix='.on').reset_index()

In [42]:
ot_res_w_on = ot_res_w_on[[
    'PamID', 'chrom.on', 'start', 'end', 'strand.on', 'genename', 'grna_seq', 'ot_seq', 'mm',
    'chrom.off', 'pos', 'strand.off', 'ref', 'alt', 'pred_cleavage_log10', 'pred_cleavage_log10.1', 'pred_cleavage_log10.2',
    'k_on_log.1', 'k_off_log.1', 'k_OI_log.1', 'k_IO_log.1', 'k_IC_log.1', 'k_CI_log.1', 'k_cat_log.1',
    'k_on_log.2', 'k_off_log.2', 'k_OI_log.2', 'k_IO_log.2', 'k_IC_log.2', 'k_CI_log.2', 'k_cat_log.2',
]]

In [45]:
ot_res_w_on.query("PamID=='PDCD1|1'").sort_values('pred_cleavage_log10', ascending=False)[['ref', 'alt', 'pred_cleavage_log10']].head()

,ref,alt,pred_cleavage_log10
237973,GGAGACTGGCCGAAGGAACCGGT,GGAGACTGGACTAAGGAACCGGA,-4.082101
237858,GGAGACTGGCCGAAGGAACCGGT,GGTGACTGGTCGAAGGAAACGAA,-4.153573
238313,GGAGACTGGCCGAAGGAACCGGT,GGAGACAGGACGAAGGAACTAGT,-4.212838
238310,GGAGACTGGCCGAAGGAACCGGT,GGAGACAGGACGAAGGAACTAGT,-4.212838
237665,GGAGACTGGCCGAAGGAACCGGT,GGGGACTAGAAGAAGGAACCAGT,-4.247734


In [46]:
ot_res_w_on.query("PamID=='PDCD1|2'").sort_values('pred_cleavage_log10', ascending=False)[['ref', 'alt', 'pred_cleavage_log10']].head()

,ref,alt,pred_cleavage_log10
177861,GGGAGACTGGCCGAAGGAACCGG,GGAAGACTGACCGAAGGAAGAGA,-2.611826
177490,GGGAGACTGGCCGAAGGAACCGG,GGGAGACTGGACGAAGGAACTTC,-3.359443
177901,GGGAGACTGGCCGAAGGAACCGG,GGAAAACTGACCGAAGGAAATGA,-3.419958
177705,GGGAGACTGGCCGAAGGAACCGG,GGGAGACTGACCAAAGGAACGAG,-3.763196
177750,GGGAGACTGGCCGAAGGAACCGG,GGAAGACAGGACGAAGGAAGAGA,-3.812833


In [47]:
ot_res_w_on.sort_values('PamID').to_csv("PDCD1-KINN_pred.txt", sep="\t", index=False)

In [50]:
%load_ext watermark
%watermark -n -u -v -iv -w

Last updated: Wed Nov 02 2022

Python implementation: CPython
Python version       : 3.8.12
IPython version      : 7.31.1

numpy     : 1.21.2
pandas    : 1.4.1
sys       : 3.8.12 (default, Oct 12 2021, 13:49:34) 
[GCC 7.5.0]
tensorflow: 2.11.0rc1
Bio       : 1.78

Watermark: 2.3.1



In [49]:
%%bash
cas-offinder

Cas-OFFinder v2.4 (Mar  5 2018)

Copyright (c) 2013 Jeongbin Park and Sangsu Bae
Website: http://github.com/snugel/cas-offinder

Usage: cas-offinder {input_file} {C|G|A}[device_id(s)] {output_file}
(C: using CPUs, G: using GPUs, A: using accelerators)

Example input file:
/var/chromosomes/human_hg19
NNNNNNNNNNNNNNNNNNNNNRG
GGCCGACCTGTCGCTGACGCNNN 5
CGCCAGCGTCAGCGACAGGTNNN 5
ACGGCGCCAGCGTCAGCGACNNN 5
GTCGCTGACGCTGGCGCCGTNNN 5

Available device list:
Type: CPU, ID: 0, <pthread-Intel(R) Xeon(R) Gold 6234 CPU @ 3.30GHz> on <Portable Computing Language>
